# **04.Results**

Summary:


1.   Import data
2.   Define functions
3.   Testing




---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import sys
sys.path.append('/content/drive/My Drive/Università/inforet_prj/')

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import pickle
import string
import seaborn as sns; sns.set()
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
from nltk import ngrams
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

## **1. Import data**

In [2]:
# ONLY ONE MODEL
with open("/content/drive/MyDrive/Università/inforet_prj/models.pkl", "rb") as f:
    models = pickle.load(f)

In [3]:
with open("/content/drive/MyDrive/Università/inforet_prj/models_time.pkl", "rb") as f:
    models_time = pickle.load(f)

In [4]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_matrix_1_pmi.pkl", "rb") as f:
    pmi_matrix_1_pmi = pickle.load(f)

In [5]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_matrix_2_pmi.pkl", "rb") as f:
    pmi_matrix_2_pmi = pickle.load(f)

In [6]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_op_matrix.pkl", "rb") as f:
    pmi_op_matrix = pickle.load(f)

In [7]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_1_pmi_bigr_matrix.pkl", "rb") as f:
    pmi_1_pmi_bigr_matrix = pickle.load(f)

In [8]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_2_pmi_bigr_matrix.pkl", "rb") as f:
    pmi_2_pmi_bigr_matrix = pickle.load(f)

## **2. Define functions**

In [9]:
def most_similar_list_schedule(model_, narcos, word, n, value=False):
    lista = []
    for sched, model in model_.items():
        if sched == narcos:
            for x in model.wv.most_similar(positive=word)[:n]:
                if value:
                    lista.append([x[0],round(x[1], 2)])
                    #print(sched, x[0],round(x[1], 2))
                else:
                    lista.append(x[0])
                    #print(sched, x[0])
    return lista

In [10]:
def most_similar_list_time(model_, time, word, n, value=False):
    lista = []
    for year, model in model_.items():
        if year == time:
            for x in model.wv.most_similar(positive=word)[:n]:
                if value:
                    lista.append([x[0],round(x[1], 2)])
                    #print(year, x[0],round(x[1], 2))
                else:
                    lista.append([x[0]])
                    #print(year, x[0])
    return lista

In [11]:
def similarity_2_words_time(model_, word1, word2):
    print('(' , word1 , '-' , word2, ')')
    for year, model in model_.items():
        try:
            print(year, round(model.wv.similarity( word1, word2 ), 3) )
        except:
            print(year, 'No data to display')
            #pass

In [12]:
def get_co_pmi_value(word1, word2, matrix):
    return matrix.loc[matrix.index == word1, [word2]]

In [13]:
def pmi(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df_pmi = df / expected
    # Silence distracting warnings about log(0):
    with np.errstate(divide='ignore'):
        df_pmi = np.log(df_pmi)
    df_pmi[np.isinf(df_pmi)] = 0.0  # log(0) = 0
    if positive:
        df_pmi[df_pmi < 0] = 0.0
    return df_pmi

## **3. Testing**

In [15]:
most_similar_list_schedule(models, 'narco_2', 'drug', 10, True)

[['narcotic', 0.8],
 ['cocaine', 0.72],
 ['buying', 0.5],
 ['amphetamine', 0.49],
 ['occasional', 0.48],
 ['consummate', 0.46],
 ['ongoing', 0.46],
 ['dope', 0.46],
 ['profit', 0.46],
 ['substance', 0.44]]

In [16]:
most_similar_list_time(models_time, '60s', 'drug', 10, True)

[['dispense', 0.83],
 ['dispensing', 0.8],
 ['selling', 0.74],
 ['possess', 0.74],
 ['transport', 0.73],
 ['unlawfully', 0.72],
 ['trade', 0.72],
 ['profess', 0.71],
 ['user', 0.7],
 ['compound', 0.7]]

In [17]:
similarity_2_words_time(models_time, 'drug','narcotic')

( drug - narcotic )
50s 0.973
60s 0.583
70s 0.486
80s 0.649
90s 0.803
00s 0.756


In [18]:
get_co_pmi_value('drug', 'crime', pmi_op_matrix)

,crime
drug,0.195879


In [19]:
get_co_pmi_value('drug', 'crime', pmi_matrix_1_pmi)

,crime
drug,0.065747


In [20]:
get_co_pmi_value('drug', 'crime', pmi_matrix_2_pmi)

,crime
drug,0.07536
